# Tutorial 6: Importing custom beam patterns.
In this tutorial, we will demonstrate how to import and use custom beam patterns. This feature was developed for simulating the propagation of measured beam patterns, but can in principle import any beam pattern in .txt format.
We shall demonstrate the custom beam pattern functionality using the same simulation as in tutorial 4: backward propagation.

In [1]:
#%matplotlib widget
# Uncomment for interactive plots when running the notebook!
# Requires ipympl package to be installed

import numpy as np

from PyPO.System import System
from PyPO.Enums import FieldComponents, Units

s = System()
s.setCustomBeamPath("custom_beam", append=True)

2026-02-24 17:19:19 - WARNING - System override set to True. 


In addition to the usual import and declaration of the System, we also call the s.setCustomBeamPath() method. This method sets the path to the custom beam. In this case, we set it to "custom_beam" and also pass "append=True". By default, the custom beam pattern is looked for in the current working directory. If we pass "append=False", `PyPO` will look if there is a directory named "custom_beam" in the absolute path from home, which is not correct. 

The custom beam should be saved in the following format: the real part as r<name_of_beam>.txt, imaginary part as i<name_of_beam>.txt. In this case, our custom beam is named 'PyPO' and is generated by converting a .png containing the word 'PyPO' to a numpy array, representing the intensity pattern. The imaginary part is set to zero.

In [7]:
source = {
        "name"      : "source",
        "gmode"     : "xy",
        "lims_x"    : np.array([-100, 100])*Units.MM,
        "lims_y"    : np.array([-100, 100])*Units.MM,
        "gridsize"  : np.array([301, 301])
        }

plane_up = {
        "name"      : "plane_up",
        "gmode"     : "xy",
        "lims_x"    : np.array([-100, 100])*Units.MM,
        "lims_y"    : np.array([-100, 100])*Units.MM,
        "gridsize"  : np.array([501, 501]),
        "flip"      : True
        }

plane_down = {
        "name"      : "plane_down",
        "gmode"     : "xy",
        "lims_x"    : np.array([-100, 100])*Units.MM,
        "lims_y"    : np.array([-100, 100])*Units.MM,
        "gridsize"  : np.array([301, 301])
        }

s.addPlane(source)

s.addPlane(plane_up)
s.addPlane(plane_down)

s.readCustomBeam("PyPO", "source", FieldComponents.Ex, lam=1)

check = np.array([0., 0., 100.])#*Units.MM
print(type(check[0]))
print(type(Units.MM.value[0]))
print(np.array([0., 0., 100.])*Units.MM)

s.translateGrids("plane_up", np.array([0, 0, 100])*Units.MM)

s.plotBeam2D("PyPO", FieldComponents.Ex)

2026-02-24 17:24:04 - INFO - Added plane source to system. 
2026-02-24 17:24:04 - INFO - Added plane plane_up to system. 
2026-02-24 17:24:04 - INFO - Added plane plane_down to system. 
<class 'numpy.float64'>
<class 'float'>
[0.0 0.0 100.0]
<class 'float'>
<class 'numpy.float64'>


UFuncTypeError: Cannot cast ufunc 'add' output from dtype('O') to dtype('float64') with casting rule 'same_kind'

We read the custom beam by calling the `s.readCustomBeam()` method. We pass the name of the beam as first argument. The second argument is the surface on which the custom beam should be defined. It is important that the gridsize of the surface equals the gridsize of the beam pattern. The third argument is the component, which we set to "Ex", the x-component of the electric field. Lastly, we pass the wavelength of the custom beam which we set at 1 mm.

In [ ]:
runPODict = {
        "t_name"    : "plane_up",
        "s_current" : "PyPO",
        "exp"       : "fwd",
        "mode"      : "JMEH",
        "name_JM"   : "JM_up",
        "name_EH"   : "EH_up"
        }

runPODict_bwd = {
        "t_name"    : "plane_down",
        "s_current" : "JM_up",
        "exp"       : "bwd",
        "mode"      : "JMEH",
        "name_JM"   : "JM_down",
        "name_EH"   : "EH_down"
        }

s.runPO(runPODict)

s.runPO(runPODict_bwd)

In [ ]:
s.plotBeam2D("EH_up", FieldComponents.Ex, vmin=-30)

In [ ]:
s.plotBeam2D("EH_down", FieldComponents.Ex, vmin=-30)